In [1]:
# imports
from scisample.samplers import new_sampler
import plotly.graph_objects as go
from scipy.optimize import minimize
import numpy as np
import yaml
import pandas as pd
import tempfile
import shutil
import os

In [2]:
# functions
def rosenbrock_vector(x):
    """ Rosenbrock function 2D """
    return (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2

def rosenbrock_xy(x, y):
    """ Rosenbrock function 2D """
    return (1 - x)**2 + 100*(y - x**2)**2

def rastrigin(x):
    """ Rastrigin function """
    n = len(x)
    return 10 * n + sum(x**2 - 10 * np.cos(2 * np.pi * x))

def ackley(x):
    """ Ackley function """
    n = len(x)
    return -20 * np.exp(-0.2 * np.sqrt((1/n) * np.sum(x**2))) - \
           np.exp((1/n) * np.sum(np.cos(2 * np.pi * x))) + 20 + np.exp(1)

def booth(x):
    """ Booth's function """
    return (x[0] + 2*x[1] - 7)**2 + (2*x[0] + x[1] - 5)**2

def beale(x):
    """ Beale's function """
    return (1.5 - x[0] + x[0]*x[1])**2 + (2.25 - x[0] + x[0]*x[1]**2)**2 + \
           (2.625 - x[0] + x[0]*x[1]**3)**2

def booth_xy(x, y):
    """ Booth's function """
    return (x + 2*y - 7)**2 + (2*x + y - 5)**2

def beale_xy(x, y):
    """ Beale's function """
    return (1.5 - x + x*y)**2 + (2.25 - x + x*y**2)**2 + \
           (2.625 - x + x*y**3)**2


def yaml_string_to_dict(yaml_string):
    yaml_data = yaml.safe_load(yaml_string)
    return yaml_data

In [3]:
import random
import numpy as np


def latin_hypercube_sampler(num_samples, num_variables, variable_ranges):
    """
    Generate a Latin Hypercube Sample with different min/max values for each variable.

    :param num_samples: Number of samples to generate.
    :param num_variables: Number of variables.
    :param variable_ranges: Dictionary specifying the min/max ranges for each variable.
                            Format: {'variable_name': (min_value, max_value), ...}
    :return: Latin Hypercube Sample as a 2D numpy array.
    """
    # Generate a random permutation for each variable
    permutation = [random.sample(range(num_samples), num_samples) for _ in range(num_variables)]

    # Create an empty array to store the Latin Hypercube Sample
    lhs = np.empty((num_samples, num_variables))

    # Generate the Latin Hypercube Sample
    for i, variable_name in enumerate(variable_ranges.keys()):
        min_value, max_value = variable_ranges[variable_name]
        for j in range(num_samples):
            lhs[j, i] = (permutation[i][j] + random.uniform(0, 1)) / num_samples
            lhs[j, i] = lhs[j, i] * (max_value - min_value) + min_value

    return lhs

num_samples = 10
num_variables = 2
variable_ranges = {'X': (0, 1), 'Y': (5, 10)}

sample = latin_hypercube_sampler(num_samples, num_variables, variable_ranges)
print(sample)

import plotly.graph_objects as go

# Generate Latin Hypercube Sample
num_samples = 10
num_variables = 2
variable_ranges = {'X': (0, 1), 'Y': (5, 10)}
sample = latin_hypercube_sampler(num_samples, num_variables, variable_ranges)

# Extract X and Y values from the sample
X = sample[:, 0]
Y = sample[:, 1]

# Create a scatter plot using Plotly
fig = go.Figure(data=go.Scatter(
    x=X,
    y=Y,
    mode='markers',
    marker=dict(
        color='red',
        size=8,
        symbol='circle'
    )
))

# Add axis labels and title
fig.update_layout(
    xaxis=dict(title='X'),
    yaxis=dict(title='Y'),
    title='Latin Hypercube Sample'
)

# Display the plot
fig.show()


[[0.209858   7.89879574]
 [0.82991648 8.78799669]
 [0.64442663 7.05303714]
 [0.49434594 9.93709993]
 [0.10775572 6.74298874]
 [0.037843   5.01136377]
 [0.57639448 8.44353546]
 [0.99260725 6.04167258]
 [0.76581176 5.77943787]
 [0.37913534 9.34109011]]


In [4]:
# Initial guess for the minimum
x0 = np.array([0.0, 0.0])

# Use Nelder-Mead algorithm to find the minimum
result = minimize(rosenbrock_vector, x0, method='Nelder-Mead')

# Print the minimum value and corresponding x, y coordinates
print("Minimum value:", result.fun)
print("Minimum point:", result.x)

# Use Nelder-Mead algorithm to find the minimum
result = minimize(booth, x0, method='Nelder-Mead')

# Print the minimum value and corresponding x, y coordinates
print("Minimum value:", result.fun)
print("Minimum point:", result.x)

# Use Nelder-Mead algorithm to find the minimum
result = minimize(beale, x0, method='Nelder-Mead')

# Print the minimum value and corresponding x, y coordinates
print("Minimum value:", result.fun)
print("Minimum point:", result.x)

Minimum value: 3.6861769151759075e-10
Minimum point: [1.00000439 1.00001064]
Minimum value: 2.5039618826825238e-09
Minimum point: [1.00001846 2.99996579]
Minimum value: 5.525325548786374e-10
Minimum point: [2.99994196 0.49998485]


In [5]:
# Contour plot
x_values = np.linspace(-1, 2, 100)  # Generate x values between -1 and 2
y_values = np.linspace(-1, 2, 100)  # Generate y values between -1 and 2
X, Y = np.meshgrid(x_values, y_values)
Z = np.log(rosenbrock_xy(X, Y))

fig = go.Figure(data=
    go.Contour(
        z=Z,
        x=x_values,
        y=y_values,
        zmin=-2,  # Set z minimum value
        zmax=6,   # Set z maximum value 
    ))
fig.show()

# Contour plot
x_values = np.linspace(-1, 4, 100)  # Generate x values between -1 and 2
y_values = np.linspace(-1, 4, 100)  # Generate y values between -1 and 2
X, Y = np.meshgrid(x_values, y_values)
Z = np.log(booth_xy(X, Y))

fig = go.Figure(data=
    go.Contour(
        z=Z,
        x=x_values,
        y=y_values,
        zmin=-4,  # Set z minimum value
        zmax=6,   # Set z maximum value 
    ))
fig.show()


/var/folders/ml/68vtzhgn0pz0bddnq90khpq80000gp/T/ipykernel_16150/1922023594.py:5: RuntimeWarning:

divide by zero encountered in log



In [6]:
# Initial set of samples
downselect_ratio = 0.3
yaml_string = """
type: best_candidate
num_samples: 30
# previous_samples: samples.csv # optional
# cost_variable: Z   # required if previous_samples is provided
parameters:
    X:
        min: -1
        max: 2
    Y:
        min: -1
        max: 2
"""
sample_dictionary = yaml_string_to_dict(yaml_string)

sampler = new_sampler(sample_dictionary)
samples = sampler.get_samples()

for sample in samples:
    sample['Z'] = rosenbrock_xy(sample['X'], sample['Y'])

print(f"samples ({len(samples)}):")
df = pd.DataFrame(samples)
# Display the 5 smallest elements of 'Z'
smallest_values = df['Z'].nsmallest(5).to_list()

iterations = 20
print(f"5 smallest {iterations}: {smallest_values}")
while iterations > 0:
    # Create a temporary directory
    temp_dir = tempfile.mkdtemp()
    temp_file = os.path.join(temp_dir, "temp.csv")
    df.to_csv(temp_file)

    # next set of samples
    yaml_string = f"""
    type: best_candidate
    num_samples: 30
    previous_samples: {temp_file} # optional
    cost_variable: Z   # required if previous_samples is provided
    cost_target_oversample_ratio: 2.0 # default = BestCandidateSampler.DEFAULT_COST_TARGET_OVERSAMPLE_RATIO
    downselect_ratio: 0.3  # default = BestCandidateSampler.DEFAULT_DOWNSELECT_RATIO
    voxel_overlap: 1.0 # default = BestCandidateSampler.DEFAULT_VOXEL_OVERLAP
    cost_target: 1.0   # optional
    parameters:
        X:
            min: -1
            max: 2
        Y:
            min: -1
            max: 2
    """
    sample_dictionary = yaml_string_to_dict(yaml_string)

    sampler = new_sampler(sample_dictionary)
    samples = sampler.get_samples()
    if len (samples) == 0:
        print("no more samples")
        break

    for sample in samples:
        sample['Z'] = rosenbrock_xy(sample['X'], sample['Y'])

    print(f"samples ({len(samples)}):")
    df_new = pd.DataFrame(samples)
    df = pd.concat([df, df_new], ignore_index=True)

    smallest_values = df['Z'].nsmallest(5).to_list()
    print(f"5 smallest {iterations}: {smallest_values}")

    shutil.rmtree(temp_dir)
    iterations -= 1
    break
    
selected_rows = df[df['Z'] <= 1]
selected_rows = df

# Contour plot
x_values = np.linspace(-1, 2, 100)  # Generate x values between -1 and 2
y_values = np.linspace(-1, 2, 100)  # Generate y values between -1 and 2
X, Y = np.meshgrid(x_values, y_values)
Z = np.log(rosenbrock_xy(X, Y))

fig = go.Figure(data=[
    go.Contour(
        z=Z,
        x=x_values,
        y=y_values,
        zmin=-2,  # Set z minimum value
        zmax=6,   # Set z maximum value 
    ),
    go.Scatter(
        x=selected_rows["X"],
        y=selected_rows["Y"],
        mode="markers",
        marker=dict(
            color="red",
            size=4,
            symbol="circle"
        ),
        name="Samples"
    )
])
fig.show()


previous_samples: [1.5794597304552112, 4.652079416948845, 5.352590466954116, 9.347524571610128, 13.084442015611735]


extrema for new input_labels:  [-0.9901034  -0.97263834] [1.99821373 1.99683777]
down sampling to 30 best candidates from 300 total points.
samples (30):
5 smallest 20: [1.5794597304552112, 4.652079416948845, 5.3525904669541156, 9.347524571610128, 13.084442015611737]
distance_map defaultdict(<class 'list'>, {0: [[0.5298768234218297, 5], [0.6741270827068797, 4], [0.7523586123479761, 6], [1.4330231448131963, 8], [1.4868832383887503, 3], [1.5417813550161465, 1], [2.4536984640334394, 2], [3.468662888319419, 7]], 1: [[0.7637645473680343, 8], [0.9119171090172926, 2], [1.011904531594317, 5], [1.5264328086310326, 6], [1.5417813550161465, 0], [1.6255894500128711, 3], [1.9268815333032725, 7], [2.2159084377230265, 4]], 2: [[0.9119171090172926, 1], [1.0149644242859799, 7], [1.020675319220243, 8], [1.9238216406116095, 5], [2.438349917648325, 6], [2.4536984640334394, 0], [2.537506559030164, 3], [3.127825546740319, 4]], 3: [[1.0135794988020455, 5], [1.0796782294201428, 4], [1.4868832383887503, 0], [1

/var/folders/ml/68vtzhgn0pz0bddnq90khpq80000gp/T/ipykernel_16150/496109904.py:84: RuntimeWarning:

divide by zero encountered in log



In [12]:
# Initial set of samples
downselect_ratio = 0.3
yaml_string = """
type: best_candidate
num_samples: 30
# previous_samples: samples.csv # optional
# cost_variable: Z   # required if previous_samples is provided
parameters:
    X:
        min: -1
        max: 2
    Y:
        min: -1
        max: 2
"""
sample_dictionary = yaml_string_to_dict(yaml_string)

sampler = new_sampler(sample_dictionary)
samples = sampler.get_samples()

for sample in samples:
    sample['Z'] = rosenbrock_xy(sample['X'], sample['Y'])

print(f"samples ({len(samples)}):")
df = pd.DataFrame(samples)
# Display the 5 smallest elements of 'Z'
smallest_values = df['Z'].nsmallest(5).to_list()

iterations = 20
print(f"5 smallest {iterations}: {smallest_values}")
while iterations > 0:
    # Create a temporary directory
    temp_dir = tempfile.mkdtemp()
    temp_file = os.path.join(temp_dir, "temp.csv")
    df.to_csv(temp_file)

    # next set of samples
    yaml_string = f"""
    type: best_candidate
    num_samples: 30
    previous_samples: {temp_file} # optional
    cost_variable: Z   # required if previous_samples is provided
    cost_target: 0.1   # optional
    parameters:
        X:
            min: -1
            max: 2
        Y:
            min: -1
            max: 2
    """
    sample_dictionary = yaml_string_to_dict(yaml_string)

    sampler = new_sampler(sample_dictionary)
    samples = sampler.get_samples()
    if len (samples) == 0:
        print("no more samples")
        break

    for sample in samples:
        sample['Z'] = rosenbrock_xy(sample['X'], sample['Y'])

    print(f"samples ({len(samples)}):")
    df_new = pd.DataFrame(samples)
    df = pd.concat([df, df_new], ignore_index=True)

    smallest_values = df['Z'].nsmallest(5).to_list()
    print(f"5 smallest {iterations}: {smallest_values}")

    shutil.rmtree(temp_dir)
    iterations -= 1
    
selected_rows = df[df['Z'] <= 0.1]

# Contour plot
x_values = np.linspace(-1, 2, 100)  # Generate x values between -1 and 2
y_values = np.linspace(-1, 2, 100)  # Generate y values between -1 and 2
X, Y = np.meshgrid(x_values, y_values)
Z = np.log(rosenbrock_xy(X, Y))

fig = go.Figure(data=[
    go.Contour(
        z=Z,
        x=x_values,
        y=y_values,
        zmin=-2,  # Set z minimum value
        zmax=6,   # Set z maximum value 
    ),
    go.Scatter(
        x=selected_rows["X"],
        y=selected_rows["Y"],
        mode="markers",
        marker=dict(
            color="red",
            size=4,
            symbol="circle"
        ),
        name="Samples"
    )
])
fig.show()


previous_samples: [0.0790871380523799, 0.4862756667589175, 1.81652884427427, 3.5137418350082035, 9.33193744244108]


extrema for new input_labels:  [-0.97337258 -0.97960985] [1.98639584 1.99988084]
down sampling to 30 best candidates from 300 total points.
samples (30):
5 smallest 20: [0.07908713805237996, 0.48627566675891754, 1.81652884427427, 3.5137418350082035, 9.331937442441081]
extrema for new input_labels:  [-0.99891931 -0.71984399] [1.75938927 1.98657171]
down sampling to 30 best candidates from 297 total points.


previous_samples: [0.0557111564100231, 0.0790871380523799, 0.2646757982355093, 0.4862756667589175, 1.3388600959524035]


samples (30):
5 smallest 20: [0.055711156410023124, 0.07908713805237996, 0.26467579823550935, 0.48627566675891754, 1.3388600959524037]
extrema for new input_labels:  [-0.5808512  -0.12380749] [1.27569082 1.58052159]
down sampling to 30 best candidates from 297 total points.


previous_samples: [0.0338315076430629, 0.0557111564100231, 0.072417750368063, 0.0790871380523799, 0.0899456299923278]


...

previous_samples: [0.0336144960989858, 0.0338315076430629, 0.0463863715528242, 0.0557111564100231, 0.0609601222287966]


samples (30):
5 smallest 18: [0.033614496098985834, 0.03383150764306293, 0.046386371552824227, 0.055711156410023124, 0.060960122228796605]
extrema for new input_labels:  [0.56817662 0.        ] [1.26744337 1.55351701]
down sampling to 30 best candidates from 300 total points.


previous_samples: [0.0045727141822858, 0.0074655803240319, 0.0121478750828128, 0.0260677338583982, 0.032456524928152]
The number of samples to keep is greater than the number of samples to generate. The number of samples to generate will be increased to the number of samples to keep.


samples (30):
5 smallest 17: [0.004572714182285822, 0.007465580324031989, 0.012147875082812851, 0.026067733858398247, 0.03245652492815202]
extrema for new input_labels:  [0.59393173 0.        ] [1.25996008 1.54833752]
down sampling to 33 best candidates from 330 total points.
samples (33):
5 smallest 16: [0.004572714182285822, 0.007465580324031989, 0.008113694510375538, 0.012147875082812851, 0.025877463921184927]
no more samples


/var/folders/ml/68vtzhgn0pz0bddnq90khpq80000gp/T/ipykernel_90869/3191965087.py:79: RuntimeWarning:

divide by zero encountered in log



In [13]:
# Initial set of samples
downselect_ratio = 0.3
yaml_string = """
type: best_candidate
num_samples: 30
# previous_samples: samples.csv # optional
# cost_variable: Z   # required if previous_samples is provided
parameters:
    X:
        min: -1
        max: 2
    Y:
        min: -1
        max: 2
"""
sample_dictionary = yaml_string_to_dict(yaml_string)

sampler = new_sampler(sample_dictionary)
samples = sampler.get_samples()

for sample in samples:
    sample['Z'] = rosenbrock_xy(sample['X'], sample['Y'])

print(f"samples ({len(samples)}):")
df = pd.DataFrame(samples)
# Display the 5 smallest elements of 'Z'
smallest_values = df['Z'].nsmallest(5).to_list()

iterations = 20
print(f"5 smallest {iterations}: {smallest_values}")
while iterations > 0:
    # Create a temporary directory
    temp_dir = tempfile.mkdtemp()
    temp_file = os.path.join(temp_dir, "temp.csv")
    df.to_csv(temp_file)

    # next set of samples
    yaml_string = f"""
    type: best_candidate
    num_samples: 30
    previous_samples: {temp_file} # optional
    cost_variable: Z   # required if previous_samples is provided
    cost_target: 0.01   # optional
    parameters:
        X:
            min: -1
            max: 2
        Y:
            min: -1
            max: 2
    """
    sample_dictionary = yaml_string_to_dict(yaml_string)

    sampler = new_sampler(sample_dictionary)
    samples = sampler.get_samples()
    if len (samples) == 0:
        print("no more samples")
        break

    for sample in samples:
        sample['Z'] = rosenbrock_xy(sample['X'], sample['Y'])

    print(f"samples ({len(samples)}):")
    df_new = pd.DataFrame(samples)
    df = pd.concat([df, df_new], ignore_index=True)

    smallest_values = df['Z'].nsmallest(5).to_list()
    print(f"5 smallest {iterations}: {smallest_values}")

    shutil.rmtree(temp_dir)
    iterations -= 1
    
selected_rows = df[df['Z'] <= 0.01]

# Contour plot
x_values = np.linspace(-1, 2, 100)  # Generate x values between -1 and 2
y_values = np.linspace(-1, 2, 100)  # Generate y values between -1 and 2
X, Y = np.meshgrid(x_values, y_values)
Z = np.log(rosenbrock_xy(X, Y))

fig = go.Figure(data=[
    go.Contour(
        z=Z,
        x=x_values,
        y=y_values,
        zmin=-2,  # Set z minimum value
        zmax=6,   # Set z maximum value 
    ),
    go.Scatter(
        x=selected_rows["X"],
        y=selected_rows["Y"],
        mode="markers",
        marker=dict(
            color="red",
            size=4,
            symbol="circle"
        ),
        name="Samples"
    )
])
fig.show()


extrema for new input_labels:  [-0.99218131 -0.99728352] [1.99617747 1.99912892]
down sampling to 30 best candidates from 300 total points.
samples (30):
5 smallest 20: [1.0648974983916997, 3.1197393930248305, 4.310031239957802, 5.908383037708051, 9.77728723308676]


previous_samples: [1.0648974983916997, 3.1197393930248305, 4.310031239957802, 5.908383037708051, 9.77728723308676]
previous_samples: [0.1274988313458017, 0.8504879948108803, 0.9276688617858038, 1.0648974983916997, 2.137116323681705]


extrema for new input_labels:  [-0.99805195 -0.1438217 ] [1.4070687  1.98886274]
down sampling to 30 best candidates from 297 total points.
samples (30):
5 smallest 20: [0.12749883134580173, 0.8504879948108803, 0.9276688617858038, 1.0648974983916997, 2.1371163236817052]
extrema for new input_labels:  [-0.78935811 -0.17410087] [1.48643763 1.95957957]
down sampling to 30 best candidates from 297 total points.


previous_samples: [0.1190359126627648, 0.1274988313458017, 0.1960754539546798, 0.2270080246498071, 0.4910976494908638]


samples (30):
5 smallest 19: [0.11903591266276484, 0.12749883134580173, 0.19607545395467982, 0.22700802464980716, 0.4910976494908638]
extrema for new input_labels:  [ 0.00653763 -0.02736671] [1.45034954 1.99411201]
down sampling to 30 best candidates from 297 total points.


...

previous_samples: [0.0003655578394999, 0.0023295278752595, 0.003288198208338, 0.0039362145899671, 0.0053575111954717]


samples (30):
5 smallest 13: [0.00036555783949995547, 0.002329527875259523, 0.003288198208338047, 0.003936214589967191, 0.005357511195471703]
extrema for new input_labels:  [0.86180075 0.        ] [1.04408139 1.09909881]
down sampling to 30 best candidates from 294 total points.


previous_samples: [0.0003655578394999, 0.0006289782416003, 0.0011457920063654, 0.0020809311112156, 0.0023295278752595]


samples (30):
5 smallest 12: [0.00036555783949995547, 0.0006289782416003322, 0.0011457920063654315, 0.0020809311112156806, 0.002329527875259523]
extrema for new input_labels:  [0.86506915 0.        ] [1.04317356 1.09459829]
down sampling to 30 best candidates from 308 total points.
samples (30):
5 smallest 11: [0.00012606063085599247, 0.00036555783949995547, 0.0006289782416003322, 0.0007075630146082485, 0.0009133411089872173]
no more samples


/var/folders/ml/68vtzhgn0pz0bddnq90khpq80000gp/T/ipykernel_90869/707124101.py:79: RuntimeWarning:

divide by zero encountered in log



In [10]:
# first random seed (downselect_number = 20, downselect_ratio = 0.3, voxel_overlap 0.6
fig = go.Figure(data=go.Scatter(x=df.index, y=df['Z'], mode='markers'))

# Set y-aZis scale to logarithmic
fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Z vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

fig = go.Figure(data=go.Scatter(x=df.index, y=df['X'], mode='markers'))

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column X vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

fig = go.Figure(data=go.Scatter(x=df.index, y=df['Y'], mode='markers'))

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Y vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

In [35]:
# first random seed (downselect_number = 20, downselect_ratio = 0.3, voxel_overlap 0.6
fig = go.Figure(data=go.Scatter(x=df.index, y=df['Z'], mode='markers'))

# Set y-aZis scale to logarithmic
fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Z vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

fig = go.Figure(data=go.Scatter(x=df.index, y=df['X'], mode='markers'))

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column X vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

fig = go.Figure(data=go.Scatter(x=df.index, y=df['Y'], mode='markers'))

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Y vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

In [33]:
# first random seed (downselect_number = 20, downselect_ratio = 0.3, voxel_overlap 0.6
fig = go.Figure(data=go.Scatter(x=df.index, y=df['Z'], mode='markers'))

# Set y-aZis scale to logarithmic
fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Z vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

fig = go.Figure(data=go.Scatter(x=df.index, y=df['X'], mode='markers'))

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column X vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

fig = go.Figure(data=go.Scatter(x=df.index, y=df['Y'], mode='markers'))

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Y vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

In [31]:
fig = go.Figure(data=go.Scatter(x=df.index, y=df['Z'], mode='markers'))

# Set y-aZis scale to logarithmic
fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Z vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

fig = go.Figure(data=go.Scatter(x=df.index, y=df['X'], mode='markers'))

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column X vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

fig = go.Figure(data=go.Scatter(x=df.index, y=df['Y'], mode='markers'))

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Y vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

In [7]:
fig = go.Figure(data=go.Scatter(x=df.index, y=df['Z'], mode='markers'))

# Set y-axis scale to logarithmic
fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Z vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()